# Processor

In [15]:
import awkward as ak
import numpy as np
import numba as nb
import uproot
import sklearn.metrics as m
import os, sys
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea.nanoevents.methods import vector
import glob

In [16]:
target = np.random.randint(low=0, high=10, size=(100, 6))  # (event, particle)
index = np.random.randint(low=-1, high=5, size=(100, 6))  # (event, particle)
builder = ak.ArrayBuilder()

for t, idx in zip(target, index):
    builder.begin_list()
    if -1 not in idx and 99 not in idx:
        for i in idx:
            builder.real(t[i])
    builder.end_list()

In [3]:
ak.sum(ak.num(builder)>0)

32

In [4]:
ak.all(index!=-1, axis=1) & ak.all(index!=99, axis=1)

<Array [True, True, False, ... False, False] type='100 * bool'>

In [17]:
@nb.njit
def inner_dim_indexing(target, index, builder):
    for t, idx in zip(target, index):
        builder.begin_list()
        if -1 not in idx and 99 not in idx:
            for i in idx:
                builder.real(t[i])
        builder.end_list()
    return builder

"""
def inner_dim_indexing(target, index):
    return target[ak.all(index!=-1, axis=1) & ak.all(index!=99, axis=1)]
"""
@nb.njit
def pos_to_bool(target, pos, builder):
    for t, p in zip(target, pos):
        builder.begin_list()
        t_len = len(t)
        p_len = len(p)
        p_sort = sorted(p)
        p_index = 0 if p_len > 0 else -1
        for i in range(t_len):
            if p_index >= 0 and p_index < p_len and i == p_sort[p_index]:
                p_index += 1
                builder.boolean(True)
            else:
                builder.boolean(False)
        builder.end_list()
    return builder

def check_clean_from_obj(target, cleanded_obj):
    comb = ak.cartesian([target, cleanded_obj], nested=True)
    dr = comb['0'].delta_r(comb['1'])
    return ak.all(dr > 0.4, axis=-1)

In [48]:
class Slimmer(processor.ProcessorABC):
    def __init__(self, out_file, is_hc=False, store_gen=True, store_pfcands=False, store_flat_only=False) -> None:
        super().__init__()
        self.out_file = out_file
        self.is_hc = is_hc
        self.store_gen = store_gen
        self.store_pfcands = store_pfcands
        self.store_flat_only = store_flat_only
        self.object = {}
        self.variable = {}


    def process_gen_leptons(self, events):
        self.pass_fiducial_gen = (events.GENlep_Hindex[:,2] != -1) & (events.GENlep_Hindex[:,2] != 99)

        _genlep_unselected = ak.zip(dict(
            pt=events.GENlep_pt,
            pt_reg=np.log(events.GENlep_pt + 1),
            eta=events.GENlep_eta,
            phi=events.GENlep_phi,
            mass=events.GENlep_mass,
            mass_reg=np.log(events.GENlep_mass + 1),
            isEle=abs(events.GENlep_id) == 11,
            isMuon=abs(events.GENlep_id) == 13,
            isTau=abs(events.GENlep_id) == 15,
            isPosCharged=events.GENlep_id > 0,
            ),
        )
        _genlep = {}
        for b in ak.fields(_genlep_unselected):
            _genlep[b] = inner_dim_indexing(ak.values_astype(_genlep_unselected[b], np.float32), events.GENlep_Hindex, ak.ArrayBuilder()).snapshot()
        _genlep = ak.zip(_genlep)

        _genlep_lo = ak.zip(
            dict(
                pt=_genlep.pt,
                eta=_genlep.eta,
                phi=_genlep.phi,
                mass=_genlep.mass,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )
        
        _genlep_lo_mask = ak.mask(_genlep_lo, self.pass_fiducial_gen)
        _gen_Z1_lo = ak.singletons(_genlep_lo_mask[:, 0] + _genlep_lo_mask[:, 1])
        _gen_Z2_lo = ak.singletons(_genlep_lo_mask[:, 2] + _genlep_lo_mask[:, 3])
        _gen_H_lo = _gen_Z1_lo + _gen_Z2_lo
        
        self.object['gen_Z1'] = ak.zip(dict(
            pt=_gen_Z1_lo.pt,
            pt_reg=np.log(_gen_Z1_lo.pt + 1),
            eta=_gen_Z1_lo.eta,
            phi=_gen_Z1_lo.phi,
            px=_gen_Z1_lo.x,
            py=_gen_Z1_lo.y,
            pz=_gen_Z1_lo.z,
            e=_gen_Z1_lo.t,
            mass=_gen_Z1_lo.mass,
            mass_reg=np.log(_gen_Z1_lo.mass + 1),
            ptrel=_gen_Z1_lo.pt / _gen_H_lo.pt,
            deta=_gen_Z1_lo.eta - _gen_H_lo.eta,
            dphi=_gen_Z1_lo.phi - _gen_H_lo.phi,
        ))
        self.object['gen_Z2'] = ak.zip(dict(
            pt=_gen_Z2_lo.pt,
            pt_reg=np.log(_gen_Z2_lo.pt + 1),
            eta=_gen_Z2_lo.eta,
            phi=_gen_Z2_lo.phi,
            px=_gen_Z2_lo.x,
            py=_gen_Z2_lo.y,
            pz=_gen_Z2_lo.z,
            e=_gen_Z2_lo.t,
            mass=_gen_Z2_lo.mass,
            mass_reg=np.log(_gen_Z2_lo.mass + 1),
            ptrel=_gen_Z2_lo.pt / _gen_H_lo.pt,
            deta=_gen_Z2_lo.eta - _gen_H_lo.eta,
            dphi=_gen_Z2_lo.phi - _gen_H_lo.phi,
        ))
        self.object['genlep'] = _genlep
        _gen_H_lo_4 = ak.concatenate([_gen_H_lo for _ in range(4)], axis=1)
        self.object['genlep']['px'] = _genlep_lo.x
        self.object['genlep']['py'] = _genlep_lo.y
        self.object['genlep']['pz'] = _genlep_lo.z
        self.object['genlep']['e'] = _genlep_lo.t
        self.object['genlep']['ptrel'] = self.object['genlep'].pt / _gen_H_lo_4.pt
        self.object['genlep']['deta'] = self.object['genlep'].eta - _gen_H_lo_4.pt
        self.object['genlep']['dphi'] = self.object['genlep'].phi - _gen_H_lo_4.phi


    def process_gen_bc_hadrons(self, events):
        _absid = abs(events.GENPart_id)
        _genparticles = ak.zip(dict(
            pt=events.GENPart_pt,
            pt_reg=np.log(events.GENPart_pt + 1),
            eta=events.GENPart_eta,
            phi=events.GENPart_phi,
            id=events.GENPart_id,
            isb=((_absid==5) | ((_absid>=50) & (_absid<60)) | ((_absid>=500) & (_absid<600)) | ((_absid>=5000) & (_absid<6000))),
            isc=((_absid==4) | ((_absid>=40) & (_absid<50)) | ((_absid>=400) & (_absid<500)) | ((_absid>=4000) & (_absid<5000))),
            isc_from_b=events.GENPart_isCFromBHad,
            is_excite=events.GENPart_isExcite,
            # use truth higgs pt...
            ptrel=events.GENPart_pt / self.object['genh'].pt[:, 0],
            deta=events.GENPart_eta - self.object['genh'].eta[:, 0],
            dphi=events.GENPart_phi - self.object['genh'].phi[:, 0],
        ))
        _genpart_cut = (_genparticles.is_excite == False)
        self.object['genpartons'] = _genparticles[_genpart_cut & (_absid < 10)]
        self.object['genhadrons'] = _genparticles[_genpart_cut & (_absid > 10)]


    def process_leptons(self, events):
        self.pass_fiducial = (events.lep_Hindex[:,2] != -1) & (events.lep_Hindex[:,2] != 99) & (ak.num(events.lep_pt) == ak.num(events.lep_id)) # not sure why the length is different..

        _df = events.mask[self.pass_fiducial]
        self.object['_lep_unselected'] = ak.zip(dict(
            pt=_df.lep_pt,
            pt_reg=np.log(_df.lep_pt + 1),
            eta=_df.lep_eta,
            phi=_df.lep_phi,
            mass=_df.lep_mass,
            mass_reg=np.log(_df.lep_mass + 1),
            isEle=abs(_df.lep_id) == 11,
            isMuon=abs(_df.lep_id) == 13,
            isTau=abs(_df.lep_id) == 15,
            isPosCharged=_df.lep_id > 0,
            ),
        )
        _lep = {}
        for b in ak.fields(self.object['_lep_unselected']):
            _lep[b] = inner_dim_indexing(ak.values_astype(self.object['_lep_unselected'][b], np.float32), events.lep_Hindex, ak.ArrayBuilder()).snapshot()
        _lep = ak.zip(_lep)

        self.object['_lep_lo'] = ak.zip(
            dict(
                pt=_lep.pt,
                eta=_lep.eta,
                phi=_lep.phi,
                mass=_lep.mass,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )

        _lep_lo_mask = ak.mask(self.object['_lep_lo'], self.pass_fiducial)
        _Z1_lo = ak.singletons(_lep_lo_mask[:, 0] + _lep_lo_mask[:, 1])
        _Z2_lo = ak.singletons(_lep_lo_mask[:, 2] + _lep_lo_mask[:, 3])
        
        _H_lo= _Z1_lo + _Z2_lo
        self.object['Z1'] = ak.zip(dict(
            pt=_Z1_lo.pt,
            pt_reg=np.log(_Z1_lo.pt + 1),
            eta=_Z1_lo.eta,
            phi=_Z1_lo.phi,
            px=_Z1_lo.x,
            py=_Z1_lo.y,
            pz=_Z1_lo.z,
            e=_Z1_lo.t,
            mass=_Z1_lo.mass,
            mass_reg=np.log(_Z1_lo.mass + 1),
            ptrel=_Z1_lo.pt / _H_lo.pt,
            deta=_Z1_lo.eta - _H_lo.eta,
            dphi=_Z1_lo.phi - _H_lo.phi,
        ))
        self.object['Z2'] = ak.zip(dict(
            pt=_Z2_lo.pt,
            pt_reg=np.log(_Z2_lo.pt + 1),
            eta=_Z2_lo.eta,
            phi=_Z2_lo.phi,
            px=_Z2_lo.x,
            py=_Z2_lo.y,
            pz=_Z2_lo.z,
            e=_Z2_lo.t,
            mass=_Z2_lo.mass,
            mass_reg=np.log(_Z2_lo.mass + 1),
            ptrel=_Z1_lo.pt / _H_lo.pt,
            deta=_Z1_lo.eta - _H_lo.eta,
            dphi=_Z1_lo.phi - _H_lo.phi,
        ))
        self.object['H'] = ak.zip(dict(
            pt=_H_lo.pt,
            pt_reg=np.log(_H_lo.pt + 1),
            eta=_H_lo.eta,
            phi=_H_lo.phi,
            px=_H_lo.x,
            py=_H_lo.y,
            pz=_H_lo.z,
            e=_H_lo.t,
            mass=_H_lo.mass,
            mass_reg=np.log(_H_lo.mass + 1),
        ))
        self.object['lep'] = _lep
        _H_lo_4 = ak.concatenate([_H_lo for _ in range(4)], axis=1)
        self.object['lep']['ptrel'] = self.object['lep'].pt / _H_lo_4.pt
        self.object['lep']['deta'] = self.object['lep'].eta - _H_lo_4.pt
        self.object['lep']['dphi'] = self.object['lep'].phi - _H_lo_4.phi
        
        self.object['_H_lo'] = _H_lo
        self.object['_Z1_lo'] = _Z1_lo
        self.object['_Z2_lo'] = _Z2_lo


    def process_jets(self, events):
        _H_lo_mask = self.object['_H_lo'].mask[ak.num(self.object['_H_lo']) > 0]
        self.variable['_H_pt'] = ak.fill_none(_H_lo_mask.pt[:, 0], np.nan)
        self.variable['_H_eta'] = ak.fill_none(_H_lo_mask.eta[:, 0], np.nan)
        self.variable['_H_phi'] = ak.fill_none(_H_lo_mask.phi[:, 0], np.nan)
        self.object['jet'] = ak.zip(dict(
            pt=events.jet_pt,
            pt_reg=np.log(events.jet_pt + 1),
            eta=events.jet_eta,
            phi=events.jet_phi,
            mass=events.jet_mass,
            mass_reg=np.log(events.jet_mass + 1),
            ptrel=events.jet_pt / self.variable['_H_pt'],
            deta=events.jet_eta - self.variable['_H_eta'],
            dphi=events.jet_phi - self.variable['_H_phi'],
            ParticleNet_b=events.jet_ParticleNet_b,
            ParticleNet_bb=events.jet_ParticleNet_bb,
            ParticleNet_c=events.jet_ParticleNet_c,
            ParticleNet_cc=events.jet_ParticleNet_cc,
            ParticleNet_uds=events.jet_ParticleNet_uds,
            ParticleNet_g=events.jet_ParticleNet_g,
            ParticleNet_undef=events.jet_ParticleNet_undef,
            ParticleNet_pu=events.jet_ParticleNet_pu,
            ParticleNet_CvsL=events.jet_ParticleNet_CvsL,
            ParticleNet_CvsB=events.jet_ParticleNet_CvsB,
            ),
        )
        self.object['jet_lo'] = ak.zip(
            dict(
                pt=self.object['jet'].pt,
                eta=self.object['jet'].eta,
                phi=self.object['jet'].phi,
                mass=self.object['jet'].mass,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )
        self.object['jet']['px'] = self.object['jet_lo'].x
        self.object['jet']['py'] = self.object['jet_lo'].y
        self.object['jet']['pz'] = self.object['jet_lo'].z
        self.object['jet']['e'] = self.object['jet_lo'].t
        self.object['cleanedjet'] = self.object['jet'][events.jet_iscleanH4l]
        ## Starting from v3: add pT > 15 cut
        self.object['cleanedjet'] = self.object['cleanedjet'][self.object['cleanedjet'].pt > 15]
        # jet['isCleaned'] = pos_to_bool(events.jet_pt, events.jet_iscleanH4l, ak.ArrayBuilder()).snapshot()


    def process_SVs(self, events):
        self.object['sv'] = ak.zip(dict(
            pt=events.sv_pt,
            pt_reg=np.log(events.sv_pt + 1),
            eta=events.sv_eta,
            phi=events.sv_phi,
            mass=events.sv_mass,
            mass_reg=np.log(events.sv_mass + 1),
            ptrel=events.sv_pt / self.variable['_H_pt'],
            deta=events.sv_eta - self.variable['_H_eta'],
            dphi=events.sv_phi - self.variable['_H_phi'],
            ParticleNet_b=events.sv_ParticleNet_b,
            ParticleNet_bb=events.sv_ParticleNet_bb,
            ParticleNet_c=events.sv_ParticleNet_c,
            ParticleNet_cc=events.sv_ParticleNet_cc,
            ParticleNet_unmat=events.sv_ParticleNet_unmat,
            ),
        )
        self.object['sv_lo'] = ak.zip(
            dict(
                pt=self.object['sv'].pt,
                eta=self.object['sv'].eta,
                phi=self.object['sv'].phi,
                mass=self.object['sv'].mass,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )
        self.object['sv']['px'] = self.object['sv_lo'].x
        self.object['sv']['py'] = self.object['sv_lo'].y
        self.object['sv']['pz'] = self.object['sv_lo'].z
        self.object['sv']['e'] = self.object['sv_lo'].t

        # clean from all tight leptons
        isoCutMu, isoCutEl = 0.35, 9999.
        passed_idiso = ((abs(events.lep_id)==13) & (events.lep_RelIso<=isoCutMu)) | ((abs(events.lep_id)==11) & (events.lep_RelIso<=isoCutEl))
        _lep_unselected_passed_idiso = self.object['_lep_unselected'][passed_idiso]
        tightleps_unselected_lo = ak.zip(
            dict(
                pt=_lep_unselected_passed_idiso.pt,
                eta=_lep_unselected_passed_idiso.eta,
                phi=_lep_unselected_passed_idiso.phi,
                mass=_lep_unselected_passed_idiso.mass,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )
        sv_is_iso_all_tightleps = check_clean_from_obj(self.object['sv_lo'], tightleps_unselected_lo)

        # clean from all higgs cands leptons
        sv_is_iso_all_candleps = check_clean_from_obj(self.object['sv_lo'], self.object['_lep_lo'])

        # clean from all FSR photons matched with tight leps
        fsrph_lo = ak.zip(
            dict(
                pt=events.fsrPhotons_pt,
                eta=events.fsrPhotons_eta,
                phi=events.fsrPhotons_phi,
                mass=ak.zeros_like(events.fsrPhotons_pt),
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiMLorentzVector',
        )
        _fsrph_matched_lep_tightid = events.lep_tightId[events.fsrPhotons_lepindex]
        _fsrph_matched_lep_iso = events.lep_RelIsoNoFSR[events.fsrPhotons_lepindex]
        _fsrph_matched_lep_id = events.lep_id[events.fsrPhotons_lepindex]
        fsrph_is_qual = (
            (_fsrph_matched_lep_tightid==1) & (
                (abs(_fsrph_matched_lep_id)==13) & (_fsrph_matched_lep_iso<=isoCutMu)) | 
                ((abs(_fsrph_matched_lep_id)==11) & (_fsrph_matched_lep_iso<=isoCutEl)
            )
        )
        fsrph_qual_lo = fsrph_lo.mask[self.pass_fiducial][fsrph_is_qual] # should pass fiducial because fsrPhotons_pt/eta filled in more strict condition
        sv_is_iso_all_fsrphs = check_clean_from_obj(self.object['sv_lo'], fsrph_qual_lo)

        # define isCleaned SV
        sv_is_cleaned = (ak.fill_none(sv_is_iso_all_tightleps, True)) & sv_is_iso_all_candleps & (ak.fill_none(sv_is_iso_all_fsrphs, True))
        self.object['cleanedsv'] = self.object['sv'][sv_is_cleaned]
        self.object['sv']['isCleaned'] = sv_is_cleaned


    def process_pfcands(self, events):
        self.object['pfcand'] = ak.zip(dict(
            puppiw=events.pfcand_puppiw,
            pt=events.pfcand_pt,
            e=events.pfcand_e,
            eta=events.pfcand_eta,
            phi=events.pfcand_phi,
            pt_reg=np.log(events.pfcand_pt + 1),
            e_reg=np.log(events.pfcand_e + 1),
            ptrel=events.pfcand_pt / self.variable['_H_pt'],
            deta=events.pfcand_eta - self.variable['_H_eta'],
            dphi=events.pfcand_phi - self.variable['_H_phi'],

            hcalFrac=events.pfcand_hcalFrac,
            VTX_ass=events.pfcand_VTX_ass,
            lostInnerHits=events.pfcand_lostInnerHits,
            quality=events.pfcand_quality,
            charge=events.pfcand_charge,
            isEl=events.pfcand_isEl,
            isMu=events.pfcand_isMu,
            isChargedHad=events.pfcand_isChargedHad,
            isGamma=events.pfcand_isGamma,
            isNeutralHad=events.pfcand_isNeutralHad,
            drminsv=events.pfcand_drminsv,
            normchi2=events.pfcand_normchi2,
            dz=events.pfcand_dz,
            dzsig=events.pfcand_dzsig,
            dxy=events.pfcand_dxy,
            dxysig=events.pfcand_dxysig,
            dptdpt=events.pfcand_dptdpt,
            detadeta=events.pfcand_detadeta,
            dphidphi=events.pfcand_dphidphi,
            dxydxy=events.pfcand_dxydxy,
            dzdz=events.pfcand_dzdz,
            dxydz=events.pfcand_dxydz,
            dphidxy=events.pfcand_dphidxy,
            dlambdadz=events.pfcand_dlambdadz,
            )
        )
        pfcand_lo = ak.zip(
            dict(
                pt=self.object['pfcand'].pt,
                eta=self.object['pfcand'].eta,
                phi=self.object['pfcand'].phi,
                e=self.object['pfcand'].e,
            ),
            behavior=vector.behavior,
            with_name='PtEtaPhiELorentzVector',
        )
        self.object['pfcand']['px'] = pfcand_lo.x
        self.object['pfcand']['py'] = pfcand_lo.y
        self.object['pfcand']['pz'] = pfcand_lo.z
        self.object['pfcand'] = self.object['pfcand'][events.pfcand_pt > 0] # remove invalid candidates


    def output(self, events):
        out_tree = {}
        out_tree.update({
            'Run': events.Run,
            'Event': events.Event,
            'LumiSect': events.LumiSect,
            'genWeight': events.genWeight,
            'mass4l': events.mass4l,
            'D_bkg_kin': events.D_bkg_kin,
            'is_hc': ak.zeros_like(events.Run, dtype=bool) + self.is_hc,
            'pass_fiducial': self.pass_fiducial,
        })
        if self.store_gen and not self.store_pfcands:
            out_tree['pass_fiducial_gen'] = self.pass_fiducial_gen

        if not self.store_flat_only:
            if self.store_gen and not self.store_pfcands:
                out_tree['GENH'] = self.object['genh']
                out_tree['GENZ1'] = self.object['gen_Z1']
                out_tree['GENZ2'] = self.object['gen_Z2']
                out_tree['GENlep'] = self.object['genlep']
            if self.store_gen:
                out_tree['GENparton'] = self.object['genpartons']
                out_tree['GENhadron'] = self.object['genhadrons']
            out_tree['H'] = self.object['H']
            out_tree['Z1'] = self.object['Z1']
            out_tree['Z2'] = self.object['Z2']
            out_tree['lep'] = self.object['lep']
            out_tree['jet'] = self.object['jet']
            out_tree['cleanedjet'] = self.object['cleanedjet']
            out_tree['sv'] = self.object['sv']
            out_tree['cleanedsv'] = self.object['cleanedsv']
            if self.store_pfcands:
                out_tree['pfcand'] = self.object['pfcand']
        else:
            H_mask = self.object['H'].mask[ak.num(self.object['H']) > 0]
            Z1_mask = self.object['Z1'].mask[ak.num(self.object['Z1']) > 0]
            Z2_mask = self.object['Z2'].mask[ak.num(self.object['Z2']) > 0]
            lep_mask = self.object['lep'].mask[ak.num(self.object['lep']) == 4]

            for b in ak.fields(self.object['H']):
                out_tree[f'H_{b}'] = ak.fill_none(H_mask[b][:, 0], np.nan)
            for b in ak.fields(self.object['Z1']):
                out_tree[f'Z1_{b}'] = ak.fill_none(Z1_mask[b][:, 0], np.nan)
            for b in ak.fields(self.object['Z2']):
                out_tree[f'Z2_{b}'] = ak.fill_none(Z2_mask[b][:, 0], np.nan)
            for b in ak.fields(self.object['lep']):
                for i in range(4):
                    out_tree[f'lep{i}_{b}'] = ak.fill_none(lep_mask[b][:, i], np.nan)

            out_tree['n_cleanedjet'] = ak.num(self.object['cleanedjet'])
            cleanedjet_inds = ak.singletons(ak.argmax(self.object['cleanedjet'].ParticleNet_c + self.object['cleanedjet'].ParticleNet_cc, axis=-1))
            cleanedjet_leadc = self.object['cleanedjet'].mask[ak.num(self.object['cleanedjet']) > 0][cleanedjet_inds][:, 0]
            for b in ak.fields(self.object['cleanedjet']):
                out_tree[f'c_{b}'] = ak.fill_none(cleanedjet_leadc[b], np.nan)
            
            out_tree['n_cleanedsv'] = ak.num(self.object['cleanedsv'])
            cleanedsv_inds = ak.singletons(ak.argmax(self.object['cleanedsv'].ParticleNet_c + self.object['cleanedsv'].ParticleNet_cc, axis=-1))
            cleanedsv_leadc = self.object['cleanedsv'].mask[ak.num(self.object['cleanedsv']) > 0][cleanedsv_inds][:, 0]
            for b in ak.fields(self.object['cleanedsv']):
                out_tree[f'cleanedsv_leadc2c_{b}'] = ak.fill_none(cleanedsv_leadc[b], np.nan)
            
            # jet/sv + H/Z features (features added in v3)
            cleaned_jet_lo = self.object['jet_lo'][events.jet_iscleanH4l]
            cleaned_jet_lo = cleaned_jet_lo[cleaned_jet_lo.pt > 15]
            cleanedjet_leadc_lo = cleaned_jet_lo.mask[ak.num(self.object['cleanedjet']) > 0][cleanedjet_inds][:, 0]
            cleanedsv_lo = self.object['sv_lo'][self.object['sv']['isCleaned']]
            cleanedsv_leadc_lo = cleanedsv_lo.mask[ak.num(self.object['cleanedsv']) > 0][cleanedsv_inds][:, 0]
            
            for name, obj in zip(
                ['H', 'Z1', 'Z2'], [
                    self.object['_H_lo'].mask[ak.num(self.object['H']) > 0][:, 0],
                    self.object['_Z1_lo'].mask[ak.num(self.object['Z1']) > 0][:, 0],
                    self.object['_Z2_lo'].mask[ak.num(self.object['Z2']) > 0][:, 0]
                ]
            ):
                out_tree[f'c_{name}_mass'] = ak.fill_none((cleanedjet_leadc_lo + obj).mass, np.nan)
                out_tree[f'c_{name}_ptrel'] = ak.fill_none(cleanedjet_leadc_lo.pt / obj.pt, np.nan)
                out_tree[f'c_{name}_dr'] = ak.fill_none(cleanedjet_leadc_lo.delta_r(obj), np.nan)
                out_tree[f'c_{name}_deta'] = ak.fill_none(cleanedjet_leadc_lo.eta - obj.eta, np.nan)
                out_tree[f'c_{name}_dphi'] = ak.fill_none((cleanedjet_leadc_lo.phi - obj.phi + np.pi) % (2 * np.pi) - np.pi, np.nan)

                out_tree[f'cleanedsv_leadc2c_{name}_mass'] = ak.fill_none((cleanedsv_leadc_lo + obj).mass, np.nan)
                out_tree[f'cleanedsv_leadc2c_{name}_ptrel'] = ak.fill_none(cleanedsv_leadc_lo.pt / obj.pt, np.nan)
                out_tree[f'cleanedsv_leadc2c_{name}_dr'] = ak.fill_none(cleanedsv_leadc_lo.delta_r(obj), np.nan)
                out_tree[f'cleanedsv_leadc2c_{name}_deta'] = ak.fill_none(cleanedsv_leadc_lo.eta - obj.eta, np.nan)
                out_tree[f'cleanedsv_leadc2c_{name}_dphi'] = ak.fill_none((cleanedsv_leadc_lo.phi - obj.phi + np.pi) % (2 * np.pi) - np.pi, np.nan)
        
        if 'ZX' in self.out_file:
            out_tree['eventWeight'] = events.ZXWeight * events.ZXFileWeight
        else:
            out_tree['eventWeight'] = events.eventWeight
        out_tree = ak.Array(out_tree)
        out_tree = {
            key: out_tree[key][self.pass_fiducial & (ak.num(self.object['jet'], axis=1)>0) & ~ak.is_none(cleanedjet_leadc_lo)] 
            for key in out_tree.fields
        }
        
        out_dir = '/'.join(self.out_file.split('/')[:-1])
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        with uproot.recreate(self.out_file) as file:
            file['Events'] = out_tree
        self.out_tree = out_tree


    def process(self, events) -> ak.Array:
        self.events = events
        if self.store_gen:
            self.object['genh'] = ak.zip(dict(
                pt=ak.singletons(events.GENH_pt[:, -1]),
                pt_reg=np.log(ak.singletons(events.GENH_pt[:, -1]) + 1),
                eta=ak.singletons(events.GENH_eta[:, -1]),
                phi=ak.singletons(events.GENH_phi[:, -1]),
            ))

        # GEN leptons
        if self.store_gen and not self.store_pfcands:
            self.process_gen_leptons(events=events)
            
        # GEN B/C Hadrons
        if self.store_gen:
            self.process_gen_bc_hadrons(events=events)

        # Leptons
        self.process_leptons(events=events)

        # Jets
        self.process_jets(events=events)

        # SVs
        self.process_SVs(events=events)

        # pfcands
        if self.store_pfcands:
            self.process_pfcands(events=events)
            
        # write output
        self.output(events=events)

        return self.out_tree


    @property  # transform method into attribute and make it unchangable to hide _accumulator
    def accumulator(self):
        return self._accumulator


    def postprocess(self, accumulator):
        return super().postprocess(accumulator)

# Have a try!

In [49]:
ntuple_path = {
    'ggH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8/merged.root'],
    'HC_3FS': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/HPlusCharm_3FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8/merged.root'],
    'HC_4FS': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8/merged.root'],
    'HC_4FSFxFx': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8/merged.root'],
    'ggZZTo4e': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo4e_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'ggZZTo4mu': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo4mu_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'ggZZTo4tau': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo4tau_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'ggZZTo2e2mu': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo2e2mu_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'ggZZTo2e2tau': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo2e2tau_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'ggZZTo2mu2tau': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext1/GluGluToContinToZZTo2mu2tau_TuneCP5_13TeV-mcfm701-pythia8/merged.root'],
    'qqZZ': ['/data/pubfs/fudawei/cHZZ_postprocess/samples/raw/mc/2018/qqZZ/merged.root'],
    'ttH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/ttH_HToZZ_4LFilter_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8/merged.root'],
    'bbH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/bbH_HToZZTo4L_M125_TuneCP2_13TeV-jhugenv7011-pythia8/merged.root'],
    'tqH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/tqH_HToZZTo4L_M125_TuneCP5_13TeV-jhugenv7011-pythia8/merged.root'],
    'WminusH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/WminusH_HToZZTo4L_M125_TuneCP5_13TeV_powheg2-minlo-HWJ_JHUGenV7011_pythia8/merged.root'],
    'WplusH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/WplusH_HToZZTo4L_M125_TuneCP5_13TeV_powheg2-minlo-HWJ_JHUGenV7011_pythia8/merged.root'],
    'ZH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/ZH_HToZZ_4LFilter_M125_TuneCP5_13TeV_powheg2-minlo-HZJ_JHUGenV7011_pythia8/merged.root'],
    'VBFH': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main_ext2/VBF_HToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8/merged.root'],
    'ZX':  ['/home/pku/licq/cH/zz_v2/selection/notebooks/coffea_output/ZX_1.root'],
    #'data': ['/home/pku/licq/cH/zz_v2/samples/zzntuples/Data_UL18/data_ul18_all.root']
}

In [50]:
"""
run = processor.Runner(
    executor=processor.FuturesExecutor(compression=None, workers=30),
    schema=BaseSchema,
    savemetrics=True,
    xrootdtimeout=60 * 30,
    # chunksize = 100_000,
    # maxchunks = None,
)

for mc in ntuple_path:
    for i in range(len(ntuple_path[mc])):
        print(f'processing {mc} {i}')
    stats, metrics = run(
        fileset={mc: [ntuple_path[mc][i]]}, treename='Ana/passedEvents',
        processor_instance=Slimmer(
            out_file=f'../samples/slimmed/mc/2018/{mc}/{i}.root',
            store_gen=False, store_pfcands=False, store_flat_only=True
        ),
    )
"""

"\nrun = processor.Runner(\n    executor=processor.FuturesExecutor(compression=None, workers=30),\n    schema=BaseSchema,\n    savemetrics=True,\n    xrootdtimeout=60 * 30,\n    # chunksize = 100_000,\n    # maxchunks = None,\n)\n\nfor mc in ntuple_path:\n    for i in range(len(ntuple_path[mc])):\n        print(f'processing {mc} {i}')\n    stats, metrics = run(\n        fileset={mc: [ntuple_path[mc][i]]}, treename='Ana/passedEvents',\n        processor_instance=Slimmer(\n            out_file=f'../samples/slimmed/mc/2018/{mc}/{i}.root',\n            store_gen=False, store_pfcands=False, store_flat_only=True\n        ),\n    )\n"

In [52]:
for mc in ntuple_path:
    if mc!='qqZZ' and mc!='ZX': continue
    print(f'===> Start {mc}')
    for i in range(len(ntuple_path[mc])):
        _events = NanoEventsFactory.from_root(
            file=ntuple_path[mc][i], treepath='Ana/passedEvents', schemaclass=BaseSchema
        ).events()
        
        s = Slimmer(out_file=f'../samples/slimmed/mc/2018/{mc}/{i}.root', store_gen=False, store_pfcands=False, store_flat_only=True)
        r = s.process(_events)
        
    print(f'===> Finish {mc}')

===> Start qqZZ
===> Finish qqZZ
===> Start ZX
===> Finish ZX


In [18]:
a = uproot.open('../samples/slimmed/mc/2018/ggH/slimmed.root:Events')
s=a['H_pt'].array()
len(s)

In [7]:
events = NanoEventsFactory.from_root(
    file=ntuple_path['ggH'][0], treepath='Ana/passedEvents', schemaclass=BaseSchema
).events()

is_hc=False
store_gen=True
store_pfcands=False
store_flat_only=True

In [9]:
_pt = ak.singletons(df.GENH_pt[:, -1])
_pt
eta=ak.singletons(df.GENH_eta[:, -1])
eta

<Array [[-4.07], [-2.19], ... [2.67], [-1.16]] type='940000 * var * float32[para...'>